# Generating roughness elements

The standard use case for generating the roughness elements is to use a configuration file containing the necessary parameters.

These parameters account for the number of elements being generated in each direction and also the offset and spacing applied to each row.

In [1]:
import pathlib
import pprint
from cfdmod.use_cases.roughness_gen import GenerationParams

pp = pprint.PrettyPrinter()

cfg_file = pathlib.Path("./fixtures/tests/roughness_gen/roughness_params.yaml")
cfg = GenerationParams.from_file(cfg_file)

pp.pprint(cfg.model_dump())

{'N_elements_x': 7,
 'N_elements_y': 5,
 'element_params': {'height': 5.0, 'width': 5.0},
 'spacing_params': {'line_offset': 10.0,
                    'offset_direction': 'x',
                    'spacing': (20.0, 20.0)}}


It's also possible to generate the configurations through code.
This can be done by creating an instance of `GenerationParams` and using it for the next steps.

The example above can be instanciated as

In [2]:
from cfdmod.use_cases.roughness_gen import SpacingParams, ElementParams

element_params = ElementParams(height=5, width=5)

spacing_params = SpacingParams(spacing=(2, 2), line_offset=5, offset_direction="x")

manual_cfg = GenerationParams(
    N_elements_x=10, N_elements_y=10, element_params=element_params, spacing_params=spacing_params
)

## Build element and linear patterns

In order to build all the elements, linear patterns are applied.
The first one applies to create a row in the offset direction.
The second one applies to replicate the original row and apply an offset to odd index rows.

In [3]:
from cfdmod.use_cases.roughness_gen import build_single_element, linear_pattern

triangles, normals = build_single_element(cfg.element_params)

single_line_triangles, single_line_normals = linear_pattern(
    triangles,
    normals,
    direction=cfg.spacing_params.offset_direction,
    n_repeats=cfg.single_line_elements,
    spacing_value=cfg.single_line_spacing,
)

full_triangles, full_normals = linear_pattern(
    single_line_triangles,
    single_line_normals,
    direction=cfg.perpendicular_direction,
    n_repeats=cfg.multi_line_elements,
    spacing_value=cfg.multi_line_spacing,
    offset_value=cfg.spacing_params.line_offset,
)

print(
    "Single element:",
    f"Vertices count: {triangles.shape[0] * triangles.shape[1]}",
    f"Triangles count: {len(triangles)}",
)
print(
    "Single line",
    f"Vertices count: {single_line_triangles.shape[0] * single_line_triangles.shape[1]}",
    f"Triangles count: {len(single_line_triangles)}",
)
print(
    "Replicated lines",
    f"Vertices count: {full_triangles.shape[0] * full_triangles.shape[1]}",
    f"Triangles count: {len(full_triangles)}",
)

Single element: Vertices count: 6 Triangles count: 2
Single line Vertices count: 42 Triangles count: 14
Replicated lines Vertices count: 210 Triangles count: 70


## Export generated geometry

In order to use the generated geometry, the user must export the triangles and normals, using the geometry API and the STL format (triangles, normals).

In [4]:
from cfdmod.api.geometry.STL import export_stl
import pathlib

output_path = pathlib.Path("./output/roughness_gen")

export_stl(output_path / "roughness_elements.stl", full_triangles, full_normals)